In [2]:
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np


In [4]:
import cv2

# 1) Load ảnh mẫu
img = cv2.imread("id_blank.jpg")
cv2.namedWindow("template")

# 2) Định nghĩa callback
def print_pos(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at x={x}, y={y}")

# 3) Gán callback và hiện ảnh
cv2.setMouseCallback("template", print_pos)
cv2.imshow("template", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Clicked at x=147, y=154


KeyboardInterrupt: 

Clicked at x=404, y=292


In [1]:
import os
from PIL import ImageFont, Image, ImageDraw


# 2) load template and draw boxes
img  = Image.open("id_blank.jpg").convert("RGB")
draw = ImageDraw.Draw(img)

# 3) load fonts
font_path = "/Library/Fonts/Arial.ttf"
font_path_bold = "/Library/Fonts/Arial Bold.ttf"
font_id  = ImageFont.truetype(font_path_bold, size=34)
font_dob = ImageFont.truetype(font_path, size=20)

# 4) sample text & positions
pos_id  = (275, 165)
pos_dob = (400, 245)
sample_id  = "023456789012"
sample_dob = "01/01/1990"

# 5) measure & draw rectangles
bbox_id  = draw.textbbox((0, 0), sample_id,  font=font_id)
w_id,  h_id  = bbox_id[2]+10 - bbox_id[0], bbox_id[3]+10 - bbox_id[1]

bbox_dob = draw.textbbox((0, 0), sample_dob, font=font_dob)
w_dob, h_dob = bbox_dob[2]+10 - bbox_dob[0], bbox_dob[3]+10 - bbox_dob[1]

# 1) Draw the sample text
draw.text(pos_id,  sample_id,  font=font_id,  fill="black")
draw.text(pos_dob, sample_dob, font=font_dob, fill="black")

# 2) Draw the annotation boxes on top
draw.rectangle(
    [pos_id,  (pos_id[0] + w_id,  pos_id[1] + h_id)],
    outline="red", width=2
)
draw.rectangle(
    [pos_dob, (pos_dob[0] + w_dob, pos_dob[1] + h_dob)],
    outline="blue", width=2
)

# 3) Show or save
img.show()


In [5]:
import os
import random
import csv
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageEnhance, ImageFilter
from faker import Faker
import string

# ───────── CONFIG ─────────
fake           = Faker("vi_VN")
TEMPLATE_PATH  = "id_blank.jpg"
OUT_DIR        = "training_siamese"
CROP_ID_DIR    = os.path.join(OUT_DIR, "crops_id")
CROP_DOB_DIR   = os.path.join(OUT_DIR, "crops_dob")
FULL_DIR    = os.path.join(OUT_DIR, "full")   
LABEL_CSV      = os.path.join(OUT_DIR, "crop_labels.csv")
NUM_IMAGES     = 15000

# font paths & sizes
FONT_ID_PATH   = "/Library/Fonts/Arial Bold.ttf"
FONT_DOB_PATH  = "/Library/Fonts/Arial.ttf"
FONT_ID_SIZE   = 34
FONT_DOB_SIZE  = 20

# fixed text anchors
POS_ID  = (275, 165)
POS_DOB = (400, 245)

# make dirs
os.makedirs(OUT_DIR,    exist_ok=True)
os.makedirs(CROP_ID_DIR,  exist_ok=True)
os.makedirs(CROP_DOB_DIR, exist_ok=True)
os.makedirs(FULL_DIR,    exist_ok=True) 

# prepare CSV
with open(LABEL_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["crop_filename", "label", "field"])  # field: "ID" or "DOB"

# load fonts
font_id  = ImageFont.truetype(FONT_ID_PATH, FONT_ID_SIZE)
font_dob = ImageFont.truetype(FONT_DOB_PATH, FONT_DOB_SIZE)

for i in range(NUM_IMAGES):
    # ─── SAMPLE ───
    digits  = [str(i) for i in range(10)]
    id_num = "".join(random.choices(digits, k=12))

 # how many preferred digits must appear in the string

    def biased_dob(min_age=16, max_age=100):
        while True:
            dob = fake.date_of_birth(minimum_age=min_age, maximum_age=max_age).strftime("%d/%m/%Y")
            return dob
    import random

    def fake_dob_like():
        """
        Returns a string in DD/MM/YYYY format,
        where each digit is uniformly sampled from 0–9.
        The structure looks like a date, but the values aren't necessarily valid.
        """
        s = ''.join(random.choices(string.digits, k=8))
        return f"{s[:2]}/{s[2:4]}/{s[4:]}"


    dob = fake_dob_like()
    jitter_id  = (POS_ID[0]  + random.randint(0,0),
                   POS_ID[1]  + random.randint(0,0))
    jitter_dob = (POS_DOB[0] + random.randint(0,0),
                   POS_DOB[1] + random.randint(0,0))

    img  = Image.open(TEMPLATE_PATH).convert("RGB")
    draw = ImageDraw.Draw(img)

    draw.text(jitter_id,  id_num, font=font_id,  fill="black")
    draw.text(jitter_dob, dob,    font=font_dob, fill="black")

    # gaussian noise
    arr   = np.array(img).astype(np.float32)
    noise = np.random.normal(2, 10, size=arr.shape)
    img   = Image.fromarray(np.clip(arr + noise, 0, 255).astype(np.uint8))

    # brightness & contrast
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.6,1.3))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.8,1.2))

    # apply blur 
    blur_radius = random.uniform(0.5, 2.0)  # adjust range as needed
    img = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))
    # apply random rotation
    do_translate = (random.random() < 0.1)
    if do_translate:  # 10% chance to rotate
        angle = random.randint(-1, 1)  # small rotation
        img = img.rotate(angle, resample=Image.BICUBIC, expand=True)
    
    import random
    from PIL import Image

    def random_color_tint(img: Image.Image,
                        tints=None,
                        p: float = 0.5,
                        alpha_range: tuple = (0.2, 0.5)) -> Image.Image:
        """
        With probability p, blend one random tint over the image.
        tints: list of RGB tuples you want to include.
            Default covers yellow, blue, orange, teal, purple-blue.
        alpha_range: (min, max) blending strength.
        """
        if random.random() > p:
            return img

        if tints is None:
            tints = [
                (255, 255, 200),  # pale yellow
                (200, 220, 255),  # pale blue
                (255, 200, 160),  # pale orange
                ( 205, 255, 250),  # teal
            ]

        tint_color = random.choice(tints)
        overlay    = Image.new("RGB", img.size, tint_color)
        alpha      = random.uniform(*alpha_range)
        return Image.blend(img, overlay, alpha)
    # ─── COLOR TINT ───

    img = random_color_tint(img, p=0.7, alpha_range=(0.1, 0.5))


    # ─── MOTION BLUR ───
    if random.random() < 0.2:
        k      = random.choice([3,5])
        kernel = np.zeros((k,k), dtype=np.float32)
        kernel[k//2, :] = np.ones(k, dtype=np.float32)/k
        arr    = np.array(img)
        img    = Image.fromarray(cv2.filter2D(arr, -1, kernel))


    # ─── COMPUTE BOXES ───
    draw = ImageDraw.Draw(img)  # rebind after augment
    
    # … then when computing your boxes …
    box_id  = draw.textbbox(jitter_id,  id_num,  font=font_id)
    box_dob = draw.textbbox(jitter_dob, dob,     font=font_dob)

    def pad_box_random_id(box):
        x0, y0, x1, y1 = box
        if do_translate:
            pad_left   = random.randint(5,10)
            pad_right  = random.randint(5,10)
            pad_top    = random.randint(8,10)
            pad_bottom = random.randint(8,10)
        else:
            if random.random() < 0.1:
                pad_left   = random.randint(1,2)
                pad_right  = random.randint(1,2)
                pad_top    = random.randint(1,2)
                pad_bottom = random.randint(1,2)
            else:
                pad_left   = random.randint(5,10)
                pad_right  = random.randint(5,30)
                pad_top    = random.randint(5,10)
                pad_bottom = random.randint(5,10)
        # apply and clamp
        x0n = max(x0 - pad_left, 0)
        y0n = max(y0 - pad_top, 0)
        x1n = min(x1 + pad_right, img.width)
        y1n = min(y1 + pad_bottom, img.height)
        return (x0n, y0n, x1n, y1n)
    
    def pad_box_random_dob(box):
        x0, y0, x1, y1 = box
        if do_translate:
            pad_left   = random.randint(5,10)
            pad_right  = random.randint(5,10)
            pad_top    = random.randint(8,10)
            pad_bottom = random.randint(8,10)
        else:
            if random.random() < 0.1:
                pad_left   = random.randint(1,3)
                pad_right  = random.randint(1,3)
                pad_top    = random.randint(1,3)
                pad_bottom = random.randint(1,3)
            else:
                pad_left   = random.randint(2,7)
                pad_right  = random.randint(2,7)
                pad_top    = random.randint(2,7)
                pad_bottom = random.randint(2,7)
        # apply and clamp
        x0n = max(x0 - pad_left, 0)
        y0n = max(y0 - pad_top, 0)
        x1n = min(x1 + pad_right, img.width)
        y1n = min(y1 + pad_bottom, img.height)
        return (x0n, y0n, x1n, y1n)
    
    padded_id_box  = pad_box_random_id(box_id)
    padded_dob_box = pad_box_random_dob(box_dob)
    do_translate = False  # reset for next image
    full_fn = f"full_{i:05d}.jpg"
    img.save(os.path.join(FULL_DIR, full_fn))
    # ─── CROP & SAVE ───
    # ID crop
    crop_id = img.crop(padded_id_box)
    fn_id   = f"id_{i:05d}.jpg"
    crop_id.save(os.path.join(CROP_ID_DIR, fn_id))
    # DOB crop
    crop_dob = img.crop(padded_dob_box)
    fn_dob   = f"dob_{i:05d}.jpg"
    crop_dob.save(os.path.join(CROP_DOB_DIR, fn_dob))
    
    # ─── APPEND LABELS ───
    with open(LABEL_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([fn_id, id_num, "ID"])
        writer.writerow([fn_dob, dob, "DOB"])

print(f"Done! Cropped {NUM_IMAGES} IDs and DOBs in:\n"
      f"  • {CROP_ID_DIR}\n  • {CROP_DOB_DIR}\nLabels in {LABEL_CSV}")


Done! Cropped 15000 IDs and DOBs in:
  • training_siamese/crops_id
  • training_siamese/crops_dob
Labels in training_siamese/crop_labels.csv


In [ ]:
     if random.random() < 0.6:
    # pick warm (yellow) or cool (blue)
        if random.random() < 0.3:
            tint_color = (255, 255, 200)   # pale yellow
        elif random.random() < 0.7:
            tint_color = (200, 220, 255)  # pale blue
        else:
            tint_color = (30, 200, 180)   # teal

        overlay = Image.new("RGB", img.size, tint_color)
        alpha   = random.uniform(0.2, 0.5)  # tint strength
        img     = Image.blend(img, overlay, alpha)